In [ ]:
import os
import torchvision.utils as utils
import sys
sys.path.append("../")
from model.models import *
from model.cond_refinenet_dilated import *
from functions.loss import *
from functions.utils import *

import dataset
import itertools
import matplotlib.pyplot as plt

from main import dict2namespace
import yaml

Does not make sense to use diagonal matrix parameterized by UNet

In [ ]:
sigma = 0.5
with open("../runs/logs/low_rank_mnist_0.5_rank50/config.yml".format(sigma), 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
config = dict2namespace(config)

In [ ]:
dim = 784
device = "cuda"
ckpt_path = "../runs/logs/low_rank_mnist_0.5_rank50/ckpt".format(sigma)

model = LowRankS2(config)
model = model.to(device)
model = torch.nn.DataParallel(model) # moved to below
state_dict = torch.load(os.path.join(ckpt_path, 'checkpoint.pth'), map_location=device)
model.load_state_dict(state_dict[0])
model.eval()
 
# load s1 model
s1 = CondRefineNetDilated(config).to(device)
s1 = torch.nn.DataParallel(s1)
s1.load_state_dict(state_dict[1])
s1.eval()

print("model loaded")

In [ ]:
import torchvision
import torchvision.transforms as transforms
transform = transforms.Compose([transforms.ToTensor(),])

batch_size = 100
channel = 1
image_size = img = 28

testset = torchvision.datasets.MNIST(root='../data/mnist', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

In [ ]:
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
dataiter = iter(testloader)
test_X, _ = dataiter.next()

test_X = test_X.to(device)[:10]
perturb_X = test_X + torch.randn_like(test_X) * sigma
perturb_X = perturb_X.view(perturb_X.shape[0], -1)
with torch.no_grad():
    score_2 = model(perturb_X).detach().reshape(perturb_X.shape[0], image_size ** 2, image_size ** 2) # linear model setup
    score_2 = score_2.reshape(perturb_X.shape[0], image_size ** 2, image_size ** 2)

In [ ]:
print(test_X.shape)
test_X = torch.tensor(test_X, dtype=torch.float).to(device).unsqueeze(dim=1)
imshow(torchvision.utils.make_grid(test_X.reshape(-1, 1,28,28).to("cpu"), nrow=20))

perturb_X = test_X + torch.randn_like(test_X) * sigma

perturb_X = perturb_X.view(perturb_X.shape[0], -1)
with torch.no_grad():
    score_2 = model(perturb_X).detach().reshape(perturb_X.shape[0], image_size ** 2, image_size ** 2) # linear model setup
    score_2 = score_2.reshape(perturb_X.shape[0], image_size ** 2, image_size ** 2)

In [ ]:
with torch.no_grad():
    covariance = score_2 * (sigma ** 4) + torch.eye(28 ** 2, device=device).unsqueeze(dim=0) * (sigma ** 2)
covariance.shape

In [ ]:
cov_diagonal = torch.diagonal(covariance, dim1=-2, dim2=-1).view(covariance.shape[0], channel, image_size, image_size)
plt.figure(figsize=(10,10))
imshow(torchvision.utils.make_grid(cov_diagonal.reshape(-1, 1,28,28).to("cpu")*10, nrow=len(cov_diagonal)))